In [1]:
!sudo apt update
!sudo apt install libgmp-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Fetched 2,351 kB in 2s (1,477 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be 

CPU:

In [2]:
import numpy as np
from numba import cuda
from concurrent.futures import ThreadPoolExecutor

def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def modular_arithmetic_sieve(start, end, threshold):
    small_primes = [p for p in range(2, threshold) if is_prime(p)]
    candidates = []

    for p in range(start, end + 1):
        if not is_prime(p):
            continue

        mersenne_num = 2 ** p - 1
        is_candidate = True

        for prime in small_primes:
            if mersenne_num % prime == 0:
                is_candidate = False
                break

        if is_candidate:
            candidates.append(p)

    return candidates

def lucas_lehmer_test(p):
    s = 4
    m = 2 ** p - 1

    for _ in range(p - 2):
        s = pow(s, 2, m) - 2

    return s == 0

def test_mersenne_prime(exp):
    if lucas_lehmer_test(exp):
        return exp
    return None

# Example usage
start_exponent = 2
end_exponent = 1000
threshold = 100

candidate_exponents = modular_arithmetic_sieve(start_exponent, end_exponent, threshold)
print("Candidate Mersenne prime exponents after modular arithmetic sieve:", candidate_exponents)

with ThreadPoolExecutor() as executor:
    mersenne_primes = list(executor.map(test_mersenne_prime, candidate_exponents))
    mersenne_primes = [p for p in mersenne_primes if p is not None]

print("Mersenne prime exponents after Lucas-Lehmer test:", mersenne_primes)

Candidate Mersenne prime exponents after modular arithmetic sieve: [7, 13, 17, 19, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
Mersenne prime exponents after Lucas-Lehmer test: [7, 13, 17, 19, 31, 61, 89, 107, 127, 521, 607]


CPU Multithreaded:

In [3]:
import math
import multiprocessing

def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def modular_arithmetic_sieve(start, end, threshold):
    small_primes = [p for p in range(2, threshold) if is_prime(p)]
    candidates = []

    for p in range(start, end + 1):
        if not is_prime(p):
            continue

        mersenne_num = 2 ** p - 1
        is_candidate = True

        for prime in small_primes:
            if mersenne_num % prime == 0:
                is_candidate = False
                break

        if is_candidate:
            candidates.append(p)

    return candidates

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        base = (base * base) % modulus
        exponent //= 2
    return result

def lucas_lehmer_test(p):
    m = 2 ** p - 1
    s = 4

    for _ in range(p - 2):
        s = modular_pow(s, 2, m) - 2
        if s < 0:
            s += m

    return s == 0

def test_mersenne_prime(exp):
    if lucas_lehmer_test(exp):
        return exp
    return None

def find_mersenne_primes(start_exponent, end_exponent, threshold):
    candidate_exponents = modular_arithmetic_sieve(start_exponent, end_exponent, threshold)
    print("Candidate Mersenne prime exponents after modular arithmetic sieve:", candidate_exponents)

    with multiprocessing.Pool() as pool:
        mersenne_primes = pool.map(test_mersenne_prime, candidate_exponents)
        mersenne_primes = [p for p in mersenne_primes if p is not None]

    print("Mersenne prime exponents after Lucas-Lehmer test:", mersenne_primes)

# Example usage
start_exponent = 2
end_exponent = 1000
threshold = 100

find_mersenne_primes(start_exponent, end_exponent, threshold)

Candidate Mersenne prime exponents after modular arithmetic sieve: [7, 13, 17, 19, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
Mersenne prime exponents after Lucas-Lehmer test: [7, 13, 17, 19, 31, 61, 89, 107, 127, 521, 607]
